In [105]:
import torch
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

print(device)

cuda:0


In [106]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

from sklearn.cluster import DBSCAN
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMRegressor
from catboost import CatBoostClassifier

import pandas as pd
from category_encoders import OneHotEncoder, TargetEncoder

import warnings

# 경고 무시
warnings.filterwarnings("ignore", category=FutureWarning)

import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

# 한글 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'  # 예시로 'NanumGothic'을 사용

In [89]:
import random
import os

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

In [90]:
train = pd.read_csv("../data/train.csv")
test = pd.read_csv("../data/test.csv")

In [91]:
train = train.drop(['ID'], axis=1)
test = test.drop(['ID'], axis=1)

In [92]:
train

,Age,Gender,Education_Status,Employment_Status,Working_Week (Yearly),Industry_Status,Occupation_Status,Race,Hispanic_Origin,Martial_Status,...,Citizenship,Birth_Country,Birth_Country (Father),Birth_Country (Mother),Tax_Status,Gains,Losses,Dividends,Income_Status,Income
0,63,M,Middle (7-8),Full-Time,4,Social Services,Services,White,All other,Married,...,Native,US,US,US,Nonfiler,0,0,0,Unknown,425
1,37,M,Associates degree (Vocational),Full-Time,52,Entertainment,Services,White,All other,Separated,...,Native,US,US,US,Single,0,0,0,Under Median,0
2,58,F,High graduate,Full-Time,52,Manufacturing (Non-durable),Admin Support (include Clerical),Black,All other,Married,...,Native,US,US,US,Married Filling Jointly both under 65 (MFJ),3411,0,0,Under Median,860
3,44,M,High graduate,Full-Time,52,Retail,Technicians & Support,White,All other,Divorced,...,Native,US,US,US,Single,0,0,0,Under Median,850
4,37,F,High graduate,Full-Time,52,Retail,Sales,White,All other,Divorced,...,Native,US,US,US,Head of Household (HOH),0,0,0,Unknown,570
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,33,M,High graduate,Children or Armed Forces,52,Manufacturing (Durable),Handlers/Cleaners,White,All other,Single,...,Native,US,US,US,Single,0,0,0,Under Median,1300
19996,20,F,College,Full-Time,12,Education,Admin Support (include Clerical),White,Mexican-American,Single,...,Native,US,Mexico,Mexico,Nonfiler,0,0,0,Under Median,850
19997,22,M,College,Children or Armed Forces,52,Transportation,Technicians & Support,White,All other,Single,...,Native,US,US,US,Single,0,0,0,Unknown,999
19998,76,F,High graduate,Not Working,0,Not in universe or children,Unknown,White,All other,Widowed,...,Native,US,Scotland,England,Single,0,0,0,Under Median,0


In [93]:
test['Household_Status'].fillna('Nonfamily householder', inplace=True)

In [94]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 22 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Age                     20000 non-null  int64 
 1   Gender                  20000 non-null  object
 2   Education_Status        20000 non-null  object
 3   Employment_Status       20000 non-null  object
 4   Working_Week (Yearly)   20000 non-null  int64 
 5   Industry_Status         20000 non-null  object
 6   Occupation_Status       20000 non-null  object
 7   Race                    20000 non-null  object
 8   Hispanic_Origin         20000 non-null  object
 9   Martial_Status          20000 non-null  object
 10  Household_Status        20000 non-null  object
 11  Household_Summary       20000 non-null  object
 12  Citizenship             20000 non-null  object
 13  Birth_Country           20000 non-null  object
 14  Birth_Country (Father)  20000 non-null  object
 15  Bi

In [95]:
# 원-핫 인코딩을 적용할 열 지정
one_hot_cols = ['Gender', 'Income_Status','Tax_Status','Citizenship']

# 원-핫 인코더 생성 및 train 데이터에 적용
one_hot_encoder = OneHotEncoder(cols=one_hot_cols, use_cat_names=True)
train_encoded = one_hot_encoder.fit_transform(train[one_hot_cols])

# 동일한 원-핫 인코더를 test 데이터에 적용
test_encoded = one_hot_encoder.transform(test[one_hot_cols])

In [96]:
# 레이블 인코더 객체 생성
label_encoder = LabelEncoder()

# one_hot_cols에 명시된 열을 제외한 카테고리형 변수를 인코딩 대상으로 설정
encoding_target = [col for col in train.columns if train[col].dtype == 'object' and col not in one_hot_cols]

# 인코딩 대상 열에 대해 레이블 인코딩 적용
for col in encoding_target:
    # train과 test 데이터셋에서 해당 열의 모든 값을 문자열로 변환
    test_encoded[col] = train[col].astype(str)
    test_encoded[col] = test[col].astype(str)

    # 레이블 인코더를 train 데이터에 적합시키고 변환
    label_encoder.fit(train[col])
    train_encoded[col] = label_encoder.transform(train[col])

    # test 데이터에도 동일한 인코더를 적용
    test_encoded[col] = label_encoder.transform(test[col])

In [97]:
train_encoded = pd.concat([train_encoded, train[['Working_Week (Yearly)','Age','Gains','Losses','Income']]], axis=1)
test_encoded = pd.concat([test_encoded, test[['Working_Week (Yearly)','Age','Gains','Losses']]], axis=1)

train = train_encoded
test = test_encoded

In [98]:
train = train.drop(train[train['Gains'] > 60000].index)

train['working_week_0'] = (train['Working_Week (Yearly)']<10).astype(int)
test['working_week_0'] = (test['Working_Week (Yearly)']<10).astype(int)

test['working_week_45']=(test['Working_Week (Yearly)']>45).astype(int)
test['working_week_45']=(test['Working_Week (Yearly)']>45).astype(int)

In [55]:
# # 로그 변환을 적용할 열 목록
# columns_to_log_transform = ['Working_Week (Yearly)', 'Age', 'Gains', 'Losses', 'Income']
# 
# # 각 열에 대해 로그 변환 적용
# for column in columns_to_log_transform:
#     train[column] = np.log1p(train[column])
# 
# columns_to_log_transform_test = ['Working_Week (Yearly)', 'Age', 'Gains', 'Losses']
# 
# for column in columns_to_log_transform_test:
#     test[column] = np.log1p(test[column])



In [56]:
train

,Gender_M,Gender_F,Income_Status_Unknown,Income_Status_Under Median,Income_Status_Over Median,Tax_Status_Nonfiler,Tax_Status_Single,Tax_Status_Married Filling Jointly both under 65 (MFJ),Tax_Status_Head of Household (HOH),Tax_Status_Married Filling Jointly one over 65 & one under 65 (MFJ),...,Birth_Country,Birth_Country (Father),Birth_Country (Mother),Working_Week (Yearly),Age,Gains,Losses,Income,working_week_0,working_week_45
0,1,0,1,0,0,1,0,0,0,0,...,39,39,39,4,63,0,0,425,1,0
1,1,0,0,1,0,0,1,0,0,0,...,39,39,39,52,37,0,0,0,0,1
2,0,1,0,1,0,0,0,1,0,0,...,39,39,39,52,58,3411,0,860,0,1
3,1,0,0,1,0,0,1,0,0,0,...,39,39,39,52,44,0,0,850,0,1
4,0,1,1,0,0,0,0,0,1,0,...,39,39,39,52,37,0,0,570,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,1,0,0,1,0,0,1,0,0,0,...,39,39,39,52,33,0,0,1300,0,1
19996,0,1,0,1,0,1,0,0,0,0,...,39,25,25,12,20,0,0,850,0,0
19997,1,0,1,0,0,0,1,0,0,0,...,39,39,39,52,22,0,0,999,0,1
19998,0,1,0,1,0,0,1,0,0,0,...,39,34,8,0,76,0,0,0,1,0


In [57]:
test

,Gender_M,Gender_F,Income_Status_Unknown,Income_Status_Under Median,Income_Status_Over Median,Tax_Status_Nonfiler,Tax_Status_Single,Tax_Status_Married Filling Jointly both under 65 (MFJ),Tax_Status_Head of Household (HOH),Tax_Status_Married Filling Jointly one over 65 & one under 65 (MFJ),...,Martial_Status,Household_Status,Household_Summary,Birth_Country,Birth_Country (Father),Birth_Country (Mother),Working_Week (Yearly),Age,Gains,Losses
0,1,0,0,1,0,0,1,0,0,0,...,5,18,4,39,40,40,0,79,0,0
1,1,0,0,1,0,1,0,0,0,0,...,5,2,0,39,39,39,0,47,0,0
2,0,1,0,1,0,0,1,0,0,0,...,5,2,0,39,39,39,52,18,0,0
3,0,1,1,0,0,0,0,1,0,0,...,1,30,7,39,39,39,30,39,0,0
4,1,0,1,0,0,1,0,0,0,0,...,5,7,2,39,39,39,0,6,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1,0,0,1,0,0,1,0,0,0,...,5,18,4,39,39,39,52,31,0,0
9996,1,0,0,1,0,0,0,1,0,0,...,1,16,4,39,39,39,52,27,0,0
9997,1,0,0,1,0,1,0,0,0,0,...,5,2,0,39,39,39,7,18,0,0
9998,1,0,0,1,0,1,0,0,0,0,...,5,7,2,39,39,39,0,9,0,0


In [58]:
# train['Income'] = np.log1p(train['Income'])

In [59]:
X = train[train.columns.drop('Income')]
Y = train['Income']

In [60]:
# X와 Y로 나누기
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.05, random_state=42, shuffle=True)

In [61]:
import optuna
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

# 데이터셋을 미리 준비하세요
# X_train, X_test, y_train, y_test = train_test_split(...)

def objective(trial):
    # Optuna가 탐색할 파라미터 공간 정의
    param = {
        'objective': 'reg:squarederror',
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 9),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 9),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
    }

    # XGBoost 모델 초기화 및 훈련
    model = XGBRegressor(**param)
    model.fit(X_train, y_train)

    # 예측 및 RMSE 계산
    preds = model.predict(X_test)
    preds[preds < 100] = 0
    rmse = np.sqrt(mean_squared_error(y_test, preds))

    return rmse

# Optuna study 객체 생성 및 최적화 실행
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100) # n_trials: 시도할 횟수

# 최적의 파라미터 출력
print(f"Best trial: {study.best_trial.params}")

[I 2024-03-29 23:21:36,490] A new study created in memory with name: no-name-098acb09-96c2-41ea-9677-857ddd64b9e2
[I 2024-03-29 23:21:38,544] Trial 0 finished with value: 626.7743456849909 and parameters: {'learning_rate': 0.15635580177779082, 'n_estimators': 727, 'max_depth': 9, 'min_child_weight': 6, 'subsample': 0.5718119595719013, 'colsample_bytree': 0.7492333826384184}. Best is trial 0 with value: 626.7743456849909.
[I 2024-03-29 23:21:38,886] Trial 1 finished with value: 512.3669629768935 and parameters: {'learning_rate': 0.08040649096029281, 'n_estimators': 392, 'max_depth': 3, 'min_child_weight': 7, 'subsample': 0.9038857309169082, 'colsample_bytree': 0.6262879140618095}. Best is trial 1 with value: 512.3669629768935.
[I 2024-03-29 23:21:39,658] Trial 2 finished with value: 532.7931002649436 and parameters: {'learning_rate': 0.11212484494296009, 'n_estimators': 803, 'max_depth': 4, 'min_child_weight': 4, 'subsample': 0.6348264667186465, 'colsample_bytree': 0.6012836578468949}. 

Best trial: {'learning_rate': 0.05335048300623485, 'n_estimators': 326, 'max_depth': 3, 'min_child_weight': 9, 'subsample': 0.543193413822322, 'colsample_bytree': 0.6433479331784054}


In [81]:
from xgboost import XGBRegressor

# XGBRegressor 모델 초기화
xgb_model = XGBRegressor(objective='reg:squarederror',
                         learning_rate =0.05760153355463915,
                         n_estimators= 517,
                         max_depth= 3,
                         min_child_weight= 8,
                         subsample= 0.800517868378667,
                         colsample_bytree= 0.7715630487016912,
                         
)

# 모델 훈련
xgb_model.fit(X_train, y_train)

# 테스트 데이터에 대한 예측 수행
y_pred_xgb = xgb_model.predict(X_test)
y_pred_xgb[y_pred_xgb<100] = 0
# RMSE 계산
rmse_xgb = mean_squared_error(y_test, y_pred_xgb, squared=False)
print(f"XGB RMSE: {rmse_xgb}")

XGB RMSE: 512.9901477025223


In [64]:
def objective(trial):
    # Optuna가 탐색할 파라미터 공간 정의
    param = {
        'objective': 'regression',
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 9),
        'num_leaves': trial.suggest_int('num_leaves', 31, 256),  # LightGBM 특유의 파라미터
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 9),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
    }

    # LightGBM 모델 초기화 및 훈련
    model = LGBMRegressor(**param)
    model.fit(X_train, y_train, eval_set=[(X_test, y_test)])  # 조기 종료를 위한 설정 추가

    # 예측 및 RMSE 계산
    preds = model.predict(X_test)
    preds[preds < 100] = 0  # 특정 조건에 대한 후처리
    rmse = np.sqrt(mean_squared_error(y_test, preds))

    return rmse

# Optuna study 객체 생성 및 최적화 실행
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)  # n_trials: 시도할 횟수

# 최적의 파라미터 출력
print(f"Best trial: {study.best_trial.params}")

[I 2024-03-29 23:24:11,493] A new study created in memory with name: no-name-fce12d39-7dc3-43a9-930f-35ebb02438c5


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002133 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 553
[LightGBM] [Info] Number of data points in the train set: 18971, number of used features: 34
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Start training from score 556.577197
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain,

[I 2024-03-29 23:24:12,040] Trial 0 finished with value: 552.3845494125699 and parameters: {'learning_rate': 0.21515982062573485, 'n_estimators': 516, 'max_depth': 6, 'num_leaves': 251, 'min_child_weight': 4, 'subsample': 0.9247568289585262, 'colsample_bytree': 0.8824948406164708}. Best is trial 0 with value: 552.3845494125699.


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000782 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 553
[LightGBM] [Info] Number of data points in the train set: 18971, number of used features: 34
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Start training from score 556.577197
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain,

[I 2024-03-29 23:24:12,292] Trial 1 finished with value: 514.0351650130392 and parameters: {'learning_rate': 0.021338810714605576, 'n_estimators': 469, 'max_depth': 4, 'num_leaves': 153, 'min_child_weight': 4, 'subsample': 0.7666826144247822, 'colsample_bytree': 0.7367606099034297}. Best is trial 1 with value: 514.0351650130392.


...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[I 2024-03-29 23:24:42,258] Trial 98 finished with value: 514.9816715067617 and parameters: {'learning_rate': 0.07564310411779676, 'n_estimators': 566, 'max_depth': 3, 'num_leaves': 99, 'min_child_weight': 4, 'subsample': 0.7375963225659615, 'colsample_bytree': 0.6816948873615805}. Best is trial 71 with value: 512.4194332178448.


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000649 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 553
[LightGBM] [Info] Number of data points in the train set: 18971, number of used features: 34
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Start training from score 556.577197
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain,

[I 2024-03-29 23:24:42,528] Trial 99 finished with value: 515.1427940534703 and parameters: {'learning_rate': 0.05471500963759518, 'n_estimators': 550, 'max_depth': 3, 'num_leaves': 62, 'min_child_weight': 5, 'subsample': 0.7709952205390674, 'colsample_bytree': 0.6478741252170811}. Best is trial 71 with value: 512.4194332178448.


Best trial: {'learning_rate': 0.038654923859543146, 'n_estimators': 441, 'max_depth': 4, 'num_leaves': 125, 'min_child_weight': 5, 'subsample': 0.8168330627381541, 'colsample_bytree': 0.6920564591574419}


In [84]:
# XGBRegressor 모델 초기화
lgbm_model = LGBMRegressor(
                         learning_rate =0.038654923859543146,
                         n_estimators= 441,
                         max_depth= 4,
                         num_leaves= 125,
                         min_child_weight = 5,
                         subsample= 0.8168330627381541,
                         colsample_bytree= 0.6920564591574419,
                         )

# 모델 훈련
lgbm_model.fit(X_train, y_train)

# 테스트 데이터에 대한 예측 수행
y_pred_lgbm = lgbm_model.predict(X_test)
y_pred_lgbm[y_pred_lgbm<100] = 0
# RMSE 계산
rmse_lgbm = mean_squared_error(y_test, y_pred_lgbm, squared=False)
print(f"XGB RMSE: {rmse_lgbm}")

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000655 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 553
[LightGBM] [Info] Number of data points in the train set: 18971, number of used features: 34
[LightGBM] [Info] Start training from score 556.577197
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best

In [78]:
import optuna
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

# 데이터셋을 미리 준비하세요
# X_train, X_test, y_train, y_test = train_test_split(...)

def objective(trial):
    # Optuna가 탐색할 파라미터 공간 정의
    param = {
        'objective': 'RMSE',
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 9),
        'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-8, 10.0),  # L2 정규화 계수
        'bootstrap_type': 'Bernoulli',
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),  # 'Bernoulli' 부트스트랩 타입을 사용할 때 필요
        'random_seed' : 42
    }

    # CatBoost 모델 초기화 및 훈련
    model = CatBoostRegressor(**param, silent=True)  # silent=True로 설정하여 학습 중 메시지 출력 억제
    model.fit(X_train, y_train, eval_set=[(X_test, y_test)], early_stopping_rounds=10)  # 조기 종료 설정

    # 예측 및 RMSE 계산
    preds = model.predict(X_test)
    preds[preds < 100] = 0  # 특정 조건에 대한 후처리
    rmse = np.sqrt(mean_squared_error(y_test, preds))

    return rmse

# Optuna study 객체 생성 및 최적화 실행
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)  # n_trials: 시도할 횟수

# 최적의 파라미터 출력
print(f"Best trial: {study.best_trial.params}")

[I 2024-03-29 23:34:20,383] A new study created in memory with name: no-name-326697d8-7fe4-4ef5-8255-da81b6adeb4d
[I 2024-03-29 23:34:20,575] Trial 0 finished with value: 519.9567689924547 and parameters: {'learning_rate': 0.25541266352952396, 'n_estimators': 121, 'max_depth': 6, 'l2_leaf_reg': 0.008108023337434817, 'subsample': 0.5751790029910593}. Best is trial 0 with value: 519.9567689924547.
[I 2024-03-29 23:34:20,704] Trial 1 finished with value: 535.6868774885494 and parameters: {'learning_rate': 0.24866093699161543, 'n_estimators': 632, 'max_depth': 6, 'l2_leaf_reg': 1.1420831865521811e-08, 'subsample': 0.9099846491870023}. Best is trial 0 with value: 519.9567689924547.
[I 2024-03-29 23:34:21,127] Trial 2 finished with value: 524.6666490606489 and parameters: {'learning_rate': 0.02809371402019535, 'n_estimators': 150, 'max_depth': 5, 'l2_leaf_reg': 5.570706495783961, 'subsample': 0.7820363209655286}. Best is trial 0 with value: 519.9567689924547.
[I 2024-03-29 23:34:21,327] Tria

KeyboardInterrupt: 

In [85]:
# XGBRegressor 모델 초기화
cat_model = CatBoostRegressor(
    learning_rate =0.09755345353231588,
    n_estimators= 372,
    max_depth= 8,
    l2_leaf_reg = 0.018838567676701338,
    subsample= 0.5839698627283093,
    bootstrap_type = 'Bernoulli',
    objective = 'RMSE',
    silent=True,
    random_seed = 42

)

# 모델 훈련
cat_model.fit(X_train, y_train)

# 테스트 데이터에 대한 예측 수행
y_pred_cat = cat_model.predict(X_test)
y_pred_cat[y_pred_cat<100] = 0
# RMSE 계산
rmse_cat = mean_squared_error(y_test, y_pred_cat, squared=False)
print(f"XGB RMSE: {rmse_cat}")

XGB RMSE: 515.6004968952714


In [100]:
pred_final_xgb = xgb_model.predict(test)
# pred_final = np.expm1(pred_final)
pred_final_xgb[pred_final_xgb<100] = 0

pred_final_cat = cat_model.predict(test)
# pred_final = np.expm1(pred_final)
pred_final_cat[pred_final_cat<100] = 0

pred_final_lgbm = lgbm_model.predict(test)
# pred_final = np.expm1(pred_final)
pred_final_lgbm[pred_final_lgbm<100] = 0



In [102]:
import numpy as np

# pred_final_xgb, pred_final_cat, pred_final_lgbm은 NumPy 배열로 가정
# 이 배열들은 각 모델의 예측값을 담고 있음

# 세 모델의 예측값을 평균내어 최종 예측값 계산
pred_ensemble = (pred_final_xgb + pred_final_cat + pred_final_lgbm) / 3



[  0.           0.         390.96818941 ... 338.24980672   0.
 757.67472684]


In [103]:
submission = pd.read_csv("../data/sample_submission.csv")
submission['Income'] = pred_ensemble
submission

,ID,Income
0,TEST_0000,0.000000
1,TEST_0001,0.000000
2,TEST_0002,390.968189
3,TEST_0003,702.355383
4,TEST_0004,0.000000
...,...,...
9995,TEST_9995,878.722877
9996,TEST_9996,783.873666
9997,TEST_9997,338.249807
9998,TEST_9998,0.000000


In [104]:
submission.to_csv("../submission/test10.csv", index = False)